In [1]:
import pandas as pd
%matplotlib inline
import cufflinks as cf
from statsmodels.tsa.stattools import acf, pacf, kpss
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

In [2]:
entire_df = pd.read_csv('air traffic.csv')
entire_df = entire_df[['Year', "Month", "Flt"]]
entire_df["Flt"] = entire_df["Flt"].str.replace(",", "").astype(int)
entire_df['date'] = pd.to_datetime(entire_df['Year'].astype(str) + '-' + entire_df['Month'].astype(str).str.zfill(2))
entire_df.set_index('date', inplace=True)
del entire_df["Year"]
del entire_df["Month"]
cut_df = entire_df.loc["2003-01": "2006-12"]
cut_df

,Flt
date,
2003-01-01,842827
2003-02-01,741610
2003-03-01,856120
2003-04-01,821265
2003-05-01,844662
2003-06-01,856576
2003-07-01,894576
2003-08-01,894497
2003-09-01,835821


In [3]:
layout_options = {'yaxis': {'type': "linear"}}

In [4]:
cut_df["Flt"].iplot(title = "Total Flights in US Over Time", yTitle = "Count", xTitle = "Year", y = "Flt", layout_update = layout_options)

In [5]:
#Want to create a forecast for summer months of 2006. Will train on df from start of 2005 to May 2006
training_df = cut_df.loc["2005-05": "2006-05"]
training_df

,Flt
date,
2005-05-01,941186
2005-06-01,933236
2005-07-01,961902
2005-08-01,964102
2005-09-01,876747
2005-10-01,892282
2005-11-01,856725
2005-12-01,867307
2006-01-01,851600


In [6]:
#Stationarity Test with Kwiatkowski-Phillips-Schmidt-Shin Approach

kpss_test = kpss(training_df['Flt'], regression = 'c', nlags = 5) # Test itself
kpss_output = pd.Series(kpss_test[0:3], index=['Test Statistic', 'p-value', 'Lags Used']) # Creating initial series with first outputs of test
for key, value in kpss_test[3].items(): # Can iterate to populate critical value indexes and values into series
    kpss_output[f'Critical Value ({key})'] = value

kpss_output

# With test statistic value below critical values at all percentage levels, data is likely stationary
# Since data is stationary no need for differencing, d value of 0 for ARIMA model

/var/folders/q2/pyrq5k_s7gz_4v527qy_7lcc0000gn/T/ipykernel_11390/846450299.py:3: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




Test Statistic           0.289648
p-value                  0.100000
Lags Used                5.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64

In [7]:
lag_acf = acf(training_df['Flt'], nlags = 5) # Use of acf function and converting to series
lag_acf_series = pd.Series(lag_acf)
lag_acf_series.iplot(kind = "bar", yTitle = "ACF", xTitle = "Lag", title = "ACF Analysis of Airline Data")
# At lag 3 is when ACF is closest to 0, will use q value of 3 for ARIMA model

In [8]:
lag_pacf = pacf(training_df['Flt'], nlags = 3, method = "ywm") # Use of pacf function and converting to series
lag_pacf_series = pd.Series(lag_pacf)
lag_pacf_series.iplot(kind = "bar", xTitle = "Lag", yTitle = "PACF", title = 'PACF Analysis of Airline Data')
# Lag 2 is last lag spike before flattening out. Will use p value of 2 in ARIMA model

In [9]:
ARIMA_model = SARIMAX(training_df["Flt"], order = (1, 0, 0), seasonal_order = (1, 0, 0, 12), enforce_stationarity=False,
                        enforce_invertibility=False)
print(ARIMA_model.fit().summary())

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -0.00000D+00    |proj g|=  0.00000D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      0      1      0     0     0   0.000D+00  -0.000D+00
  F =  -0.0000000000000000     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
                                     SARIMAX Results                                      
Dep. Variable:                                Flt   No. Observations:                   13
Model:      

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

 This problem is unconstrained.
/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/mlemodel.py:1234: RuntimeWarning:

invalid value encountered in divide

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tools/eval_measures.py:415: RuntimeWarning:

divide by zero encountered in log

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tools/eval_measures.py:440: RuntimeWarning:

divide by zero encountered in log

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tools/eval_measures.py:440: RuntimeWarnin

In [10]:
auto_model = auto_arima(training_df["Flt"], seasonal = False, stepwise = True, suppress_warnings = True)
print(auto_model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   13
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -157.396
Date:                Wed, 12 Jun 2024   AIC                            320.792
Time:                        19:23:55   BIC                            322.487
Sample:                    05-01-2005   HQIC                           320.444
                         - 05-01-2006                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   4.723e+05   1.85e+05      2.548      0.011    1.09e+05    8.36e+05
ar.L1          0.4713      0.211      2.238      0.025       0.059       0.884
sigma2       1.86e+09      6.400   2.91e+08      0.0

In [11]:
forecast = ARIMA_model.fit().forecast(steps = 4) # Need the next 4 months of data


new_index = ["2006-06", "2006-07", "2006-08", "2006-09"]

forecast.index = new_index # Changing index and ensuring it is DateTime object
forecast.index = pd.to_datetime(forecast.index)
print(forecast)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -0.00000D+00    |proj g|=  0.00000D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      0      1      0     0     0   0.000D+00  -0.000D+00
  F =  -0.0000000000000000     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
2006-06-01    885186.322971
2006-07-01    912376.391865
2006-08-01    914463.120100
2006-09-01    831605.781502
Name: predicted_mean, dtype: float64


 This problem is unconstrained.
/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/mlemodel.py:1234: RuntimeWarning:

invalid value encountered in divide



In [12]:
arima_df = pd.DataFrame()
arima_df = entire_df.loc["2003-05": "2006-09"]
arima_df["Truncated Data"] = arima_df["Flt"].loc["2003-05": "2006-05"]
del arima_df["Flt"]
arima_df["ARIMA Prediction"] = forecast
arima_df["Real Data"] = entire_df.loc["2006-06": "2006-09"]
arima_df

,Truncated Data,ARIMA Prediction,Real Data
date,,,
2003-05-01,844662.0,NaN,NaN
2003-06-01,856576.0,NaN,NaN
2003-07-01,894576.0,NaN,NaN
2003-08-01,894497.0,NaN,NaN
2003-09-01,835821.0,NaN,NaN
2003-10-01,872580.0,NaN,NaN
2003-11-01,819659.0,NaN,NaN
2003-12-01,855970.0,NaN,NaN
2004-01-01,845530.0,NaN,NaN


In [13]:
arima_df.iplot(kind = "line", xTitle = "Count", yTitle = "Year", title = "ARIMA vs. Real Airline Data", layout_update = layout_options)

In [18]:
mse = mean_squared_error(arima_df["Real Data"].loc["2006-06": "2006-09"], arima_df["ARIMA Prediction"].loc["2006-06": "2006-09"])
mae = mean_absolute_error(arima_df["Real Data"].loc["2006-06": "2006-09"], arima_df["ARIMA Prediction"].loc["2006-06": "2006-09"])
mape = mean_absolute_percentage_error(arima_df["Real Data"].loc["2006-06": "2006-09"], arima_df["ARIMA Prediction"].loc["2006-06": "2006-09"]) * 100

In [25]:
print(f'MSE is: {mse}')
print(f'MAE is: {mae}')
print(f'MAPE is: {mape}')

MSE is: 493306391.8587953
MAE is: 20072.59589045093
MAPE is: 2.2229684199970694
